In [1]:
import sys
import os
import argparse
import shutil
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.metrics import QuantileLoss

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


# category columns
CATE_COLS = ['building_num', "mgrp", 'holiday', 'dow', 'cluster', 'hot']


# building cluster based on kmeans
CLUSTER = {
    # 0 : 건물 기타
    0: [i for i in range(1, 15 + 1)],
    # 1 : 공공
    1: [i for i in range(16, 23 + 1)],
    # 2 : 대학교
    2: [i for i in range(24, 31 + 1)],
    # 3 : 데이터센터
    3: [i for i in range(32, 36 + 1)],
    # 4 : 백화점 및 아울렛
    4: [i for i in range(37, 44 + 1)],
    # 5 : 병원
    5: [i for i in range(45, 52 + 1)],
    # 6 : 상용
    6: [i for i in range(53, 60 + 1)],
    # 7 : 아파트
    7: [i for i in range(61, 68 + 1)],
    # 8 : 연구소
    8: [i for i in range(69, 76 + 1)],
    # 9 : 지식산업센터
    9: [i for i in range(77, 84 + 1)],
    # 10 : 할인마트
    10: [i for i in range(85, 92 + 1)],
    # 11 : 호텔 및 리조트
    11: [i for i in range(93, 100 + 1)]
}

# length of training data for prediction (5 weeks)
ENCODER_LENGTH_IN_WEEKS = 5

# learning rate determined by a cv run with train data less 1 trailing week as validation
LRS = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
       0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]

# number of epochs found in cv run
NUM_EPOCHS = 60

# number of seeds to use
NUM_SEEDS = 10

BATCH_SIZE = 16 #

# hyper parameters determined by cv runs with train data less 1 trailing week as validation
PARAMS = {
    'gradient_clip_val': 0.9658579636307634,
    'hidden_size': 180,
    'dropout': 0.19610151695402608,
    'hidden_continuous_size': 90,
    'attention_head_size': 4,
    'learning_rate': 0.08
}

c:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\USER\AppData\Local\Programs\Python\Python38\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#경로만 자신의 환경에 맞게 잘 설정해주세요!
DATAROOT=''
CKPTROOT = DATAROOT+"/ckpts" # directory for model checkpoints
CSVROOT = DATAROOT+"/csvs" # directory for prediction outputs
SUBFN = DATAROOT+"/sub.csv" # final submission file path
LOGDIR = DATAROOT+"/logs" # pytorch_forecasting requirs logger

In [3]:
building_df = pd.read_csv('../../../building_info.csv', encoding='UTF8')
submission_df = pd.read_csv('../../../sample_submission.csv', encoding='UTF8')
train_df = pd.read_csv('../../../train.csv', encoding='UTF8')
test_df = pd.read_csv('../../../test.csv', encoding='UTF8')

In [4]:
current_path = os.getcwd()
parent_path = os.path.abspath(os.path.join(current_path, '..', '..'))
sys.path.append(parent_path)

In [5]:
train_df.drop(['num_date_time'],axis=1,inplace=True)
test_df.drop(['num_date_time'],axis=1,inplace=True)
test_df = pd.read_csv('../../../merge_test_encoding.csv', encoding = "CP949")
train_loc = pd.read_csv('../../../train_location.csv')
train_df = pd.concat([train_df,train_loc['location']],axis=1)

In [6]:
from preprocessing import fillnan
from preprocessing import preprocessing_all

train_df, test_df = fillnan(train_df, test_df)
train_df, test_df = preprocessing_all(train_df, test_df, building_df)
train_df['date_time'] = pd.to_datetime(train_df['date_time'])
test_df['date_time'] = pd.to_datetime(test_df['date_time'])
train_df

,num,date_time,temp,rainy,wind,hum,power,location,연면적,냉방면적,...,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,body_temp,low_power_day,power_diff_ratio,power_increase_summer
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,11,110634.00,39570.00,...,0,0,0,0,781.2,0,0,0.0,0.0,0
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,11,110634.00,39570.00,...,0,0,0,0,810.0,0,0,0.0,0.0,0
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,11,110634.00,39570.00,...,0,0,0,0,796.5,0,0,0.0,0.0,0
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,11,110634.00,39570.00,...,0,0,0,0,801.6,0,0,0.0,0.0,0
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,11,110634.00,39570.00,...,0,0,0,0,791.2,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.1,0.9,86.0,881.04,9,57497.84,40035.23,...,0,0,0,1,1986.6,0,1,0.0,0.0,0
203996,100,20220824 20,22.4,0.1,1.3,86.0,798.96,9,57497.84,40035.23,...,0,0,0,1,1926.4,0,1,0.0,0.0,0
203997,100,20220824 21,21.3,0.1,1.0,92.0,825.12,9,57497.84,40035.23,...,0,0,0,1,1959.6,0,1,0.0,0.0,0
203998,100,20220824 22,21.0,0.1,0.3,94.0,640.08,9,57497.84,40035.23,...,0,0,0,1,1974.0,0,1,0.0,0.0,0


In [8]:
def seed_all(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [9]:
# prepare train and test data
def prep(train_df,test_df):
    sz = train_df.shape[0]
    df = pd.concat([train_df, test_df])
    df["log_target"] = np.log(df.power + 1e-8)
    __ix = df.columns.get_loc('date_time')
    df['time_idx'] = (df.loc[:, 'date_time'] - df.iloc[0, __ix]).astype('timedelta64[h]').astype('int')
    train_df = df.iloc[:sz].copy()
    test_df = df.iloc[sz:].copy()
    return train_df, test_df

In [10]:
train_df, test_df = prep(train_df,test_df)

In [21]:
CATE_COLS = ['num', 'holiday', 'location','연면적', '냉방면적', '태양광용량', 'ESS저장용량', 'PCS용량',
             'body_temp',
       'low_power_day', 'power_increase_summer']

In [22]:
train_df['num'] = train_df['num'].astype(str)
test_df['num'] = test_df['num'].astype(str)
train_df['holiday'] = train_df['holiday'].astype(str)
test_df['holiday'] = test_df['holiday'].astype(str)

In [12]:
train_df['건물기타'] = train_df['건물기타'].astype(str)
test_df['건물기타'] = test_df['건물기타'].astype(str)
train_df['공공'] = train_df['공공'].astype(str)
test_df['공공'] = test_df['공공'].astype(str)
train_df['대학교'] = train_df['대학교'].astype(str)
test_df['대학교'] = test_df['대학교'].astype(str)
train_df['데이터센터'] = train_df['데이터센터'].astype(str)
test_df['데이터센터'] = test_df['데이터센터'].astype(str)
train_df['백화점및아울렛'] = train_df['백화점및아울렛'].astype(str)
test_df['백화점및아울렛'] = test_df['백화점및아울렛'].astype(str)
train_df['병원'] = train_df['병원'].astype(str)
test_df['병원'] = test_df['병원'].astype(str)
train_df['상용'] = train_df['상용'].astype(str)
test_df['상용'] = test_df['상용'].astype(str)
train_df['아파트'] = train_df['아파트'].astype(str)
test_df['아파트'] = test_df['아파트'].astype(str)
train_df['연구소'] = train_df['연구소'].astype(str)
test_df['연구소'] = test_df['연구소'].astype(str)
train_df['지식산업센터'] = train_df['지식산업센터'].astype(str)
test_df['지식산업센터'] = test_df['지식산업센터'].astype(str)
train_df['할인마트'] = train_df['할인마트'].astype(str)
test_df['할인마트'] = test_df['할인마트'].astype(str)
train_df['호텔및리조트'] = train_df['호텔및리조트'].astype(str)
test_df['호텔및리조트'] = test_df['호텔및리조트'].astype(str)

In [13]:
train_df['location'] = train_df['location'].astype(str)
test_df['location'] = test_df['location'].astype(str)

train_df['연면적'] = train_df['연면적'].astype(str)
test_df['연면적'] = test_df['연면적'].astype(str)

train_df['냉방면적'] = train_df['냉방면적'].astype(str)
test_df['냉방면적'] = test_df['냉방면적'].astype(str)

train_df['태양광용량'] = train_df['태양광용량'].astype(str)
test_df['태양광용량'] = test_df['태양광용량'].astype(str)

train_df['ESS저장용량'] = train_df['ESS저장용량'].astype(str)
test_df['ESS저장용량'] = test_df['ESS저장용량'].astype(str)

train_df['PCS용량'] = train_df['PCS용량'].astype(str)
test_df['PCS용량'] = test_df['PCS용량'].astype(str)

train_df['body_temp'] = train_df['body_temp'].astype(str)
test_df['body_temp'] = test_df['body_temp'].astype(str)

train_df['low_power_day'] = train_df['low_power_day'].astype(str)
test_df['low_power_day'] = test_df['low_power_day'].astype(str)

train_df['power_increase_summer'] = train_df['power_increase_summer'].astype(str)
test_df['power_increase_summer'] = test_df['power_increase_summer'].astype(str)

In [14]:
[os.makedirs(p, exist_ok=True) for p in (CKPTROOT, CSVROOT, LOGDIR)]

[None, None, None]

In [15]:
from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)

In [23]:
from torch.utils.data import DataLoader

def load_dataset(train_df, validate=True):
    max_encoder_length = 24 * 7 *ENCODER_LENGTH_IN_WEEKS #5
    max_prediction_length = 24 * 7

    training_cutoff = train_df['time_idx'].max()-max_prediction_length #2040 - 24*7 = 1871

    tr_ds = TimeSeriesDataSet(
      train_df[lambda x: x.time_idx <=training_cutoff] if validate else train_df,
      time_idx = "time_idx",
      target = "power",
      group_ids=["num"],
      min_encoder_length = 1,
      max_encoder_length = max_encoder_length,
      min_prediction_length=1,
      max_prediction_length=max_prediction_length,


      #Known Inputs 알고 있는 변수
      time_varying_known_categoricals = CATE_COLS,
      time_varying_known_reals=[
            "date_time",
            'hour',
            "temp",
            "wind",
            "hum",
            #"prec",
            #'cumhol',
            'sin_time', 'cos_time', 'sin_day', 'cos_day','tem_x_hum','num_day_hour_mean', 'num_day_hour_std','power_diff_ratio'
        ],
      target_normalizer=GroupNormalizer(groups=["num"], transformation="softplus"),

      #모르고 있는 변수
      time_varying_unknown_categoricals=[],
      time_varying_unknown_reals=[
            "power",
            "log_target",
            #"mean_target",
            #"mean_target_num",
            #"mean_target_mgrp",
            #"mean_target_cluster"
        ],


        add_relative_time_idx=True,  # add as feature
        add_target_scales=True,  # add as feature
        add_encoder_length=True,  # add as feature

        allow_missing_timesteps=True
    )
    ###
    validation = TimeSeriesDataSet.from_dataset(tr_ds, train_df, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=32 * 10, num_workers=12)
    ###
    '''
    va_ds = None
    if validate:
        va_ds = TimeSeriesDataSet.from_dataset(
        tr_ds, train_df, predict=True, stop_randomization=True
    )
        #va_dl = DataLoader(va_ds, batch_size=64)
    '''
    return tr_ds, val_dataloader


In [24]:
#tr_ds, va_ds = load_dataset(train_df, validate=False)
tr_ds, va_ds = load_dataset(train_df, validate=True)

In [38]:
# training
def fit(seed, tr_ds, va_loader=None):
    seed_all(seed) # doesn't really work as training is non-deterministic

    # create dataloaders for model
    tr_loader = tr_ds.to_dataloader(
        train=True, batch_size=BATCH_SIZE, num_workers=12
    )

    if va_loader is not None:
        # stop training, when loss metric does not improve on validation set
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            min_delta=1e-4,
            patience=20,
            verbose=True,
            mode="min"
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
        callbacks = [lr_logger, early_stopping_callback]
    else:
        # gather 10 checkpoints with best traing loss
        checkpoint_callback = ModelCheckpoint(
            monitor='train_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=10
        )
        callbacks = [checkpoint_callback]

    # create trainer
    trainer = pl.Trainer(
        max_epochs=2,
        devices=1,
        accelerator="gpu",
        gradient_clip_val=PARAMS['gradient_clip_val'],
        limit_train_batches=30,
        callbacks=callbacks,
        logger=TensorBoardLogger(LOGDIR)
    )

    # use pre-deterined leraning rate schedule for final submission
    learning_rate = LRS if va_loader is None else PARAMS['learning_rate']

    # initialise model with pre-determined hyperparameters
    tft = TemporalFusionTransformer.from_dataset(
        tr_ds,
        learning_rate=learning_rate,
        hidden_size=PARAMS['hidden_size'],
        attention_head_size=PARAMS['attention_head_size'],
        dropout=PARAMS['dropout'],
        hidden_continuous_size=PARAMS['hidden_continuous_size'],
        output_size=1,
        loss=SMAPE(), # SMAPE loss
        log_interval=10,  # log example every 10 batches
        logging_metrics=[SMAPE()],
        reduce_on_plateau_patience=4,  # reduce learning automatically
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

    kwargs = {'train_dataloaders': tr_loader}
    if va_loader:
        kwargs['val_dataloaders'] = va_loader

    # fit network
    trainer.fit(
        tft,
        **kwargs
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    print(f'best_model_path={best_model_path}')
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

    return best_tft

In [37]:
# training
def tune_and_fit(seed, tr_ds, va_loader):
    seed_all(seed) # doesn't really work as training is non-deterministic

    # create dataloaders for model
    tr_loader = tr_ds.to_dataloader(
        train=True, batch_size=BATCH_SIZE, num_workers=12
    )
    #val_loader = va_loader.to_dataloader(
    #    train=False, batch_size=BATCH_SIZE, num_workers=12
    #)
    
    if va_loader is not None:
        # stop training, when loss metric does not improve on validation set
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            min_delta=1e-4,
            patience=20,
            verbose=True,
            mode="min"
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
        callbacks = [lr_logger, early_stopping_callback]
    else:
        # gather 10 checkpoints with best traing loss
        checkpoint_callback = ModelCheckpoint(
            monitor='train_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=10
        )
        callbacks = [checkpoint_callback]


    
    # create study
    study = optimize_hyperparameters(
        tr_loader,
        va_loader,
        model_path="optuna_test",
        n_trials=5,
        max_epochs=30, #20
        gradient_clip_val_range=(0.01, 1.0),
        hidden_size_range=(8, 64),
        hidden_continuous_size_range=(8, 64),
        attention_head_size_range=(1, 4),
        learning_rate_range=(0.001, 0.1),
        dropout_range=(0.1, 0.3),
        trainer_kwargs=dict(limit_train_batches=100, limit_test_batches=100, limit_val_batches=100, log_every_n_steps=15, gpus=1),
        reduce_on_plateau_patience=4,
        use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
        timeout=7200,  # we can increase the timTRUEeout for better tuning.
        loss=SMAPE(),
    )
    # show best hyperparameters
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best parameters: {study.best_trial.params}")
    aa = study.best_trial.params

    #print(study.best_trial.params)
    # Retrain the full model
    #Early Stopping 
    MIN_DELTA  = 1e-4
    PATIENCE = 20

    #PL Trainer
    MAX_EPOCHS = 60   # this also one of the tuning parameters to imporve the score.
    GPUS = 1
    GRADIENT_CLIP_VAL=study.best_trial.params['gradient_clip_val']
    LIMIT_TRAIN_BATCHES=30

    #Fusion Transformer
    LR = study.best_trial.params['learning_rate']
    HIDDEN_SIZE = study.best_trial.params['hidden_size']
    DROPOUT = study.best_trial.params['dropout']
    ATTENTION_HEAD_SIZE = study.best_trial.params['attention_head_size']
    HIDDEN_CONTINUOUS_SIZE = study.best_trial.params['hidden_continuous_size']
    OUTPUT_SIZE=1
    REDUCE_ON_PLATEAU_PATIENCE=4          

    #MAX_EPOCHS = study.best_trial.params['max_epochs']
    GRADIENT_CLIP_VAL = study.best_trial.params['gradient_clip_val']
    # create trainer
    #PARAMS['gradient_clip_val'],
    trainer = pl.Trainer(
        max_epochs=MAX_EPOCHS,
        devices=1,
        accelerator="gpu",
        gradient_clip_val= GRADIENT_CLIP_VAL,       
        limit_train_batches=30,
        callbacks=callbacks,
        logger=TensorBoardLogger(LOGDIR)
    )

    # use pre-deterined leraning rate schedule for final submission
    learning_rate = LRS if va_loader is None else PARAMS['learning_rate']

    # initialise model with pre-determined hyperparameters
    tft = TemporalFusionTransformer.from_dataset(
        tr_ds,
        learning_rate=LR,
        hidden_size=HIDDEN_SIZE,
        attention_head_size=ATTENTION_HEAD_SIZE,
        dropout=DROPOUT,
        hidden_continuous_size=HIDDEN_CONTINUOUS_SIZE,
        output_size=1,
        loss=SMAPE(), # SMAPE loss
        log_interval=10,  # log example every 10 batches
        logging_metrics=[SMAPE()],
        reduce_on_plateau_patience=4,  # reduce learning automatically
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

    kwargs = {'train_dataloaders': tr_loader}
    if va_loader:
        kwargs['val_dataloaders'] = va_loader

    # fit network
    trainer.fit(
        tft,
        **kwargs
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    print(f'best_model_path={best_model_path}')
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

    return best_tft

In [ ]:
#Optuna 부분
#seed=[17, 1218, 20230725, 1998, 32, 40, 800, 6651, 4931, 18011810]
#seed=[724, 313, 9377, 9555, 126, 877, 7777, 1004, 725, 4598723]
seed=[313]
for s in seed:
    tune_and_fit(s, tr_ds, va_ds)

[I 2023-07-27 05:18:28,612] A new study created in memory with name: no-name-e3285f63-13dc-4239-9d5d-02d0d5107398
c:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
[W 2023-07-27 05:18:28,616] Trial 0 failed because of the following error: MisconfigurationException("`CUDAAccelerator` can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].")
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\USER\AppData\Local\Programs\Python\Pytho

MisconfigurationException: `CUDAAccelerator` can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

In [39]:
#seed=[17, 1218, 20230725, 1998, 32, 40, 800, 6651, 4931, 18011810]

seed=[6321, 4, 5341, 86711, 321354, 1321]
for s in seed:
    fit(s, tr_ds)


MisconfigurationException: No supported gpu backend found!

In [ ]:
# predict 1 week
def forecast(ckpt, train_df, test_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 5*24*7 and max_prediction_length == 1*24*7

    # use 5 weeks of training data at the end
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

    # get last entry from training data
    last_data = train_df.iloc[[-1]]

    # fill NA target value in test data with last values from the train dataset
    target_cols = [c for c in test_df.columns if 'target' in c]
    for c in target_cols:
        test_df.loc[:, c] = last_data[c].item()

    decoder_data = test_df

    # combine encoder and decoder data. decoder data is to be predicted
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # num_labels: mapping from 'num' categorical feature to index in new_raw_predictions['prediction']
    #             {'5': 4, '6': 6, ...}
    # new_raw_predictions['prediction'].shape = (60, 168, 1)
    num_labels = best_tft.dataset_parameters['categorical_encoders']['building_num'].classes_

    preds = new_raw_predictions['prediction'].squeeze()

    sub_df = pd.read_csv(DATAROOT+"/sample_submission.csv")

    # get prediction for each building (num)
    for n, ix in num_labels.items():
        sub_df.loc[(sub_df['num_date_time'].apply(lambda x : int(x.split('_')[0])) == int(n)), 'answer'] = preds[ix].numpy()

    # save predction to a csv file
    outfn = CSVROOT+'/'+(Path(ckpt).stem + '.csv')
    print(outfn)
    sub_df.to_csv(outfn, index=False)

In [ ]:
def ensemble(outfn):
    # get all prediction csv files
    fns = list(glob.glob(CSVROOT+"/*.csv"))
    df0 = pd.read_csv(fns[0])
    df = pd.concat([df0] + [pd.read_csv(fn).loc[:,'answer'] for fn in fns[1:]], axis=1)
    # get median of all predcitions
    df['median'] = df.iloc[:,1:].median(axis=1)
    df = df[['num_date_time', 'median']]
    df = df.rename({'median': 'answer'}, axis=1)
    # save to submission file
    df.to_csv(outfn, index=False)

# not used for final submission
def validate(seed, tr_ds, va_ds):
    va_loader = va_ds.to_dataloader(
        train=False, batch_size=BATCH_SIZE*10, num_workers=12
    )
    best_tft = fit(seed, tr_ds, va_loader)
    actuals = torch.cat([y[0] for x, y in iter(va_loader)])
    predictions = best_tft.predict(va_loader)
    smape_per_num = SMAPE(reduction="none")(predictions, actuals).mean(1)
    print(smape_per_num)
    print(smape_per_num.mean())

In [ ]:
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    forecast(p, train_df, test_df)

### FORECAST ###


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


/content/drive/MyDrive/electric/csvs/seed=724-epoch=050-train_loss=0.04.csv


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


/content/drive/MyDrive/electric/csvs/seed=724-epoch=051-train_loss=0.04.csv


...

/content/drive/MyDrive/electric/csvs/seed=724-epoch=057-train_loss=0.04.csv


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


/content/drive/MyDrive/electric/csvs/seed=724-epoch=058-train_loss=0.04.csv


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


/content/drive/MyDrive/electric/csvs/seed=724-epoch=059-train_loss=0.04.csv


In [ ]:
print("### ENSEMBLING ###")
ensemble(CSVROOT + 'submit_v21.csv')

### ENSEMBLING ###


In [ ]:
a = pd.read_csv(CSVROOT + 'submit_v21.csv')
a

,num_date_time,answer
0,1_20220825 00,1870.348633
1,1_20220825 01,1756.653076
2,1_20220825 02,1644.968323
3,1_20220825 03,1568.214905
4,1_20220825 04,1601.211426
...,...,...
16795,100_20220831 19,880.414734
16796,100_20220831 20,800.949615
16797,100_20220831 21,730.718262
16798,100_20220831 22,623.565460
